### Tunning maximum damping rate parameter $\sigma$ for SDO ABC
#### Used for Figure 3

Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from clawpack import pyclaw
import sys
from piston_Lagrangian import setup
from matplotlib import animation
from IPython.display import HTML
plt.rcParams['text.usetex'] = True
plt.rcParams['animation.ffmpeg_path'] = r'/usr/local/bin/ffmpeg'
from utils import max_error

Compute a reference solution

In [ ]:
p = 20 #Number of  periods
p_domain = 30 #Number of periods in the domain
xmax1 = p_domain*2*np.pi
N = 250 #Cells per period at final time
mx1 = p_domain*N#int(N*(xmax1/p)) #Number of grid points
M = 0.4 
piston_freq=1.
tfinal = p*2*np.pi
piston_problem1 = setup(tfinal=tfinal, xmax=xmax1, mx=mx1, M=M, CFL=0.8,limiting=1,solver_type='sharpclaw',
order=2, time_integrator='Heun', euler_RS='euler', nout=100,
piston_freq=piston_freq)
piston_problem1.verbosity = 0
piston_problem1.run()
piston_problem1.solver.__del__() #Deallocation of Fortran arrays

Define a list of maximum damping rates $\sigma$ and sponge layer lengths $\omega$

In [ ]:
list_sponge_layer_widths = np.round(np.logspace(np.log10(0.01),np.log10(10.),11),2)
#Order list
list_sponge_layer_widths.sort()
list_damping_rates = np.array([1.,10.,20.,30.,40.,50.])
print(f"Sponge layer widths: {list_sponge_layer_widths}")
print(f"Damping rates: {list_damping_rates}")

Compute a solution with SDO

In [ ]:
p = 20 #Number of  periods
p_domain = 30 #Number of periods in the domain
xmax_expected = p*2*np.pi #Maximum value of x for nontrivial solution
xmax2 = p_domain*2*np.pi
N = 250 #Cells per period at final time
mx2 = p_domain*N#int(N*(xmax1/p)) #Number of grid points
M = 0.4 
tfinal = p*2*np.pi#p*2*np.pi
start_slowing = 20*np.pi
#stop_slowing = 22*np.pi 
start_absorbing = 20*np.pi
#stop_absorbing = 22*np.pi

def solver_mask(far_field_damping_rate,sponge_layer_width):
    piston_problem2 = setup(tfinal=tfinal, xmax=xmax2, mx=mx2, M=M, CFL=0.8,limiting=1,
                            solver_type='sharpclaw', order=2, time_integrator='Heun',euler_RS='euler_slowing',
                            start_slowing = start_slowing, stop_slowing = start_slowing+sponge_layer_width*2*np.pi,
                            start_absorbing=start_absorbing, stop_absorbing=start_absorbing+sponge_layer_width*2*np.pi,
                            far_field_damping_rate=far_field_damping_rate, far_field_damping=True, nout=100,
                            sigma_damping_type="Engsig-Karup2",sigma_slowing_type="Mayer")

    piston_problem2.verbosity = 0 
    piston_problem2.run()
    piston_problem2.solver.__del__() #Deallocation of Fortran arrays
    return piston_problem2

### Running simulation

In [ ]:
error_list = []
for sponge_layer_width in list_sponge_layer_widths:
    local_error_list = []
    for far_field_damping_rate in list_damping_rates:
        piston_problem2 = solver_mask(far_field_damping_rate,sponge_layer_width)
        indx = np.where(piston_problem2.grid.x.centers<=start_slowing)
        local_error_list.append(max_error(piston_problem1,piston_problem2, indx=indx))
        print("Done with damping rate: ", far_field_damping_rate)
    error_list.append(local_error_list)
    print("Done with sponge layer width: ", sponge_layer_width)

In [ ]:
np.array(error_list)

In [ ]:
error_list_np = np.array(error_list)#.T
error_list_np.shape

In [ ]:
#save error_list_np to file
import pickle
with open("error_different_damping_rates.txt","wb") as fp:
    pickle.dump(error_list_np,fp)


In [ ]:
#Reading data
with open("error_different_damping_rates.txt","rb") as fp:
    error_list_np = pickle.load(fp)

In [ ]:
plt.close()

In [ ]:
plt.figure(figsize=(6,2),dpi=500)

marker_list = ["<",">","^","s","*","o","v"]
nice_colors = []


for i,damping_rate in enumerate(list_damping_rates):
    plt.loglog(list_sponge_layer_widths,error_list_np[:,i],label=f"{damping_rate}", marker=marker_list[i])
plt.xlabel("Sponge layer length/wavelength "+r"$(\omega/L)$", fontsize=11)
plt.ylabel("Error SDO", fontsize=11)



xtickslabels = [str(elem) for elem in np.round(list_sponge_layer_widths,3)]
xtickslabels[:3] = [f"{elem:.0e}" for elem in list_sponge_layer_widths[:3]]

# #Remove excessive ticks
#remove 7th element from list_sponge_layer_widths

#sponge_layers_no1 = np.delete(list_sponge_layer_widths,7)
plt.gca().set_xticks([])
plt.xticks(list_sponge_layer_widths)
plt.gca().set_xticks(list_sponge_layer_widths, minor=True, fontsize=10)
xtickslabels = np.round(list_sponge_layer_widths,2)
plt.gca().set_xticklabels(xtickslabels)
plt.gca().xaxis.set_tick_params(which='minor', bottom=False)

# #Remove excessive yticks
plt.gca().set_yticks([1e-3,1e-2,1e-1], minor=True, fontsize=10)


# #set xticklabels
plt.gca().set_xticklabels(xtickslabels)



#Two column legend
plt.legend(ncol=2, title="Maximum damping rate "+r"$\sigma$", fontsize=10,loc="upper right")
